### Data preparation

#### Import packages

In [53]:
# Install the libraries
import pandas as pd
import numpy as np
from pulp import *
import pulp.apis
from datetime import datetime
import time
import warnings
import yaml
from pandas.core.common import SettingWithCopyWarning
import json

In [54]:
solver_list = pulp.listSolvers(onlyAvailable=True)

In [55]:
# Ignore SettingWithCopyWarning 
warnings.simplefilter(action='ignore', category=SettingWithCopyWarning)

In [56]:
def load_config():
    try:
        with open('config_file.yaml') as f:
            global constants
            constants = yaml.safe_load(f)
    except OSError as e:
        raise e
    except Exception as e:
        raise e

In [57]:
load_config()

#### Import data

In [58]:
# Import the data
# File name is specified in config file
data_input = pd.read_excel(constants['INPUT_FILE_NAME'], sheet_name='data', engine='pyxlsb')
real_input = pd.read_excel(constants['INPUT_FILE_NAME'], sheet_name='Real', engine='pyxlsb')
bandwidth_input = pd.read_excel(constants['INPUT_FILE_NAME'], sheet_name='PS', engine='pyxlsb')
match_input = pd.read_excel(constants['INPUT_FILE_NAME'], sheet_name='match', engine='pyxlsb')

In [59]:
# Copy datasets to implement further changes
data = data_input.copy()
real = real_input.copy()
bandwidth = bandwidth_input.copy()
match = match_input.copy()

#### Create dict with variable names

In [60]:
# Create a dict of names on a spreadsheets that we are going to use
names = {"Тариф" : "cost",
        "Плечо" : "distance",
        "Дата" : "date",
        "НБ" : "origin",
        "ОУ" : "point",
        "НП" : "brand",
        "НП_ПС" : "product",
        "Объем" : "volume",
        "reg": "region"} 

---

**Reorganize the data**

In [61]:
# Rename the names in dataframes
def rename_columns(df_list, names):
    for df in df_list:
         df.rename(columns=names, inplace=True)
    return df_list

In [62]:
data, real, bandwidth, match = rename_columns([data, real, bandwidth, match], names)

In [63]:
# Check for duplicated; will raise an error if there are duplicates
assert data[data.duplicated(subset = ['point', 'origin', 'region', 'brand', 'date'])].empty == True
assert real[real.duplicated(subset = ['point', 'brand', 'date'])].empty == True
assert bandwidth[bandwidth.duplicated(subset = ['origin', 'product', 'date'])].empty == True

In [64]:
# Make subset for the concrete time period
def select_time_period(df):
    df_subset = df[df['date'] == constants['MONTH']]
    return df_subset

# Make subset for the concrete region - ONLY FOR EXPERIMENTS, IN THE FINAL VERSION ALL REGIONS GO TOGETHER
def select_region(df):
    regions_to_select = []
    
    for region_number in constants['REGION']:
        region_name = 'Регион ' + str(region_number)
        regions_to_select.append(region_name)
        
    if 'region' in df.columns:
        df = df[(df['region'].isin(regions_to_select))]
    return df
    
# Adjust datatypes. Enumerate oilbases and petrol stations      
def adjust_datatypes(df_list):
    df_new = []
    number = 0
    oilbases_dict = dict()

    for i in bandwidth['origin'].unique():
        oilbases_dict[i] = number
        number+=1
        
    for df in df_list:
        if 'origin' in df.columns:
            df = df.replace({"origin": oilbases_dict})
        if 'point' in df.columns:
            df['point'] = df['point'].map(lambda x: int(x.strip('АЗС ')))
        df_new.append(df)
        
    return df_new

In [65]:
# Apply changes to dataframes
def apply_changes_dfs(data, real, bandwidth, match):
    
    data_subset = select_time_period(data)
    real_subset = select_time_period(real)
    bandwidth_subset = select_time_period(bandwidth)

    data_subset = select_region(data_subset)

    data_subset, real_subset, bandwidth_subset = adjust_datatypes([data_subset, real_subset, bandwidth_subset])
    data_subset['cost'] = data_subset['cost'].round(4)
    
    return data_subset, real_subset, bandwidth_subset

In [66]:
data_subset, real_subset, bandwidth_subset = apply_changes_dfs(data, real, bandwidth, match)

---

**Create dictionaries for brand-products matches**

In [67]:
# Create brand-product dict
def create_brand_product_dict(df):
    match_brand_product = dict(zip(df['brand'], df['product']))
    
    match_product_brand = dict()
    for key, value in match_brand_product.items():
        match_product_brand.setdefault(value, list()).append(key)
    return match_brand_product, match_product_brand

# Create numeration dict and inverted dict
def create_numeration_dict(df, parameter):
    match_dict = dict(enumerate(df[parameter]))
    inv_dict = {v: k for k, v in match_dict.items()}
    return match_dict, inv_dict

# Create numeration dict and inverted dict
def create_prod_numeration_dict(df, parameter):
    match_dict = dict(enumerate(df[parameter].unique()))
    inv_dict = {v: k for k, v in match_dict.items()}
    return match_dict, inv_dict

In [68]:
# Create 6 dictionaries (brand-product, brand-number, product-number, and inverted)
match_brand_product, match_product_brand = create_brand_product_dict(match)
match_brand_number, match_number_brand = create_numeration_dict(match,'brand')
match_product_number, match_number_product = create_prod_numeration_dict(match,'product')

---

In [69]:
# Create name of variable
def create_var_Name(row):
    var_name = 'P' + str(int(row["point"])) + 'O' + str(int(row["origin"])) +  '_brand_' + str(int(row["brand_number"]))
    return var_name

# Reorganize the data. Select only specified columns, enumerate brands and products
def reorganize_df(df, columns_to_keep, match_number_brand, match_number_product):
    df_reorganized = df.reset_index().loc[:, columns_to_keep]
    df_reorganized['brand_number'] = df_reorganized['brand'].map(lambda x: match_number_brand.get(x))
    df_reorganized['product_number'] = df_reorganized['product'].map(lambda x: match_number_product.get(x))
    
    return df_reorganized

In [70]:
columns_to_keep = ['origin', 'point', 'brand', 'date', 'product', 'cost', 'region']
df_reorganized = reorganize_df(data_subset, columns_to_keep, match_number_brand, match_number_product)

# Create a name variable
df_reorganized['var_Name'] = df_reorganized.apply(lambda row: create_var_Name(row), axis=1)  

---

**Create demand and supply dataframes**

In [71]:
# Create demand and supply dataframes
# Demand - volume of each product needed to be transported to the point
# Supply - volume of each product available on the origin

def create_demand_supply_dfs(real, bandwidth):
    demand = real[['point', 'brand', 'product', 'volume', 'date']]
    demand.rename(columns={'volume':'demand'}, inplace=True)
    demand['brand_number'] = demand['brand'].map(lambda x: match_number_brand.get(x))
    demand['product_number'] = demand['product'].map(lambda x: match_number_product.get(x))
    demand['demand'] = demand['demand'].round(4)
    
    supply = bandwidth[['origin', 'product', 'volume', 'date']]
    supply.rename(columns={'volume':'supply'}, inplace=True)
    supply['product_number'] = supply['product'].map(lambda x: match_number_product.get(x))
    supply['supply'] = supply['supply'].round(4)
    
    return demand, supply

In [72]:
demand, supply = create_demand_supply_dfs(real_subset, bandwidth_subset)

In [73]:
# Limit demand and supply with only needed origins and points
supply = supply[supply['origin'].isin(df_reorganized.origin.unique())]
demand = demand[demand['point'].isin(df_reorganized.point.unique())]

In [74]:
# Make a copy
df_reorganized_trans = df_reorganized.copy()
print('Length of original df_reorganized: ', len(df_reorganized))

# Check for where ALL volumes equals == 0 
df_trans= supply[supply['supply'] <= 0][['origin', 'product_number']]
origins_zero_volume = set(zip(df_trans.origin, df_trans.product_number))

pairs_origins_products_to_delete = []
pairs_points_brands_to_delete = []

# Filter df_reogranized
for _, row in df_reorganized.iterrows():

    origin, point, product, brand = row['origin'], row['point'], row['product_number'], row['brand_number']
    
    if supply[(supply['origin'] == origin) & (supply['product_number'] == product)].empty:
        pairs_origins_products_to_delete.append((origin, product))

    if demand[(demand['point'] == point) & (demand['brand_number'] == brand)].empty:
        pairs_points_brands_to_delete.append((point, brand))

pairs_origins_products_to_delete = set(pairs_origins_products_to_delete)
pairs_points_brands_to_delete = set(pairs_points_brands_to_delete)

# Update origins to delete
pairs_origins_products_to_delete.update(origins_zero_volume)

df_reorganized_trans['origin_product'] = list(zip(df_reorganized_trans.origin, df_reorganized_trans.product_number))
df_reorganized_trans['point_brand'] = list(zip(df_reorganized_trans.point, df_reorganized_trans.brand_number))
  
df_reorganized_trans = df_reorganized_trans.drop(df_reorganized_trans[df_reorganized_trans['origin_product'].isin(pairs_origins_products_to_delete)].index)
df_reorganized_trans = df_reorganized_trans.drop(df_reorganized_trans[df_reorganized_trans['point_brand'].isin(pairs_points_brands_to_delete)].index)
   
        
# Reassign df
df_reorganized = df_reorganized_trans
df_reorganized = df_reorganized.reset_index().drop('index', axis=1)
print('Length of new df_reorganized: ', len(df_reorganized))

print('\nNumber of pairs of origins-product dropped: ', len(pairs_origins_products_to_delete))
print('Number of pairs of points-brands: ', len(pairs_points_brands_to_delete))

Length of original df_reorganized:  7716
Length of new df_reorganized:  5729

Number of pairs of origins-product dropped:  6
Number of pairs of points-brands:  0


### Usage of puLP

#### Define function

In [75]:
brands = df_reorganized['brand_number'].unique()

In [76]:
brand_info_map = {}

def make_brand(brand_id, name):
    return {'name': name, 'id': brand_id, 'product_transportations': {}}

In [77]:
def make_transportation(origin, point, cost, var_name):
    return {'origin': origin, 'point': point, 'cost': cost, 'var_name': var_name}

In [78]:
def add_transportation_to_brand(brand, product_id, transportation):
    if brand['product_transportations'].get(product_id) == None:
        brand['product_transportations'][product_id] = [transportation]
        return brand
    
    brand['product_transportations'].get(product_id).append(transportation)  

    return brand

In [79]:
def append_row_to_map(brand_map, row):
    brand_id = row['brand_number']
    
    if brand_map.get(brand_id) == None:
        brand_map[brand_id] = make_brand(brand_id, row['brand'])
        
    brand_item = brand_map.get(brand_id)
    product_id = row['product_number']
    
    transportation = make_transportation(row['origin'], row['point'], row['cost'], row['var_Name'])
    brand_map[brand_id] = add_transportation_to_brand(brand_item, product_id, transportation)
    
    return brand_map

for i, row in df_reorganized.iterrows():
    brand_info_map = append_row_to_map(brand_info_map, row)

In [80]:
# Make dicts for groups - either D (diesel) or P (petrol)
def make_match_dicts(match_brand_product):
    
    match_brand_group = dict()
    match_product_group = dict()

    for k,v in match_brand_product.items():
        if "ДТ" in v:
            match_brand_group[k] = "D"
            match_product_group[v] = "D"
        
        elif ("АИ" in v) | ("Аи" in v):
            match_brand_group[k] = "P"
            match_product_group[v] = "P"

    match_group_brand = dict()

    for k,v in match_brand_group.items():
        match_group_brand.setdefault(v, list()).append(k)
    
    
    match_group_product = dict()

    for k,v in match_product_group.items():
        match_group_product.setdefault(v, list()).append(k)
    
    return match_brand_group, match_product_group, match_group_brand, match_group_product

In [81]:
match_brand_group, match_product_group, match_group_brand, match_group_product = make_match_dicts(match_brand_product)

In [82]:
# Function for dividing supply between different products
def merge_supply_by_product(brand_number, match_brand_product, match_brand_number, match_product_brand, match_number_brand):
    """This function returns a set of brands' numbers that are produced from the same product"""
    brand_name = match_brand_number[brand_number]
    product_name = match_brand_product[brand_name]
    brands_overlap_names = match_product_brand[product_name]
    brands_overlap_numbers = set(match_number_brand.get(item) for item in brands_overlap_names)

    return brands_overlap_numbers

In [83]:
# Function for dividing supply between different groups
def merge_supply_by_group(brand_number, match_brand_number, match_number_brand, match_brand_group, match_group_brand):
    """This function returns a set of brands' numbers that are produced from the same product group - P or D"""
    brand_name = match_brand_number[brand_number]
    group_name = match_brand_group[brand_name]
    brands_overlap_names = match_group_brand[group_name]
    brands_overlap_numbers = set(match_number_brand.get(item) for item in brands_overlap_names)

    return brands_overlap_numbers

In [84]:
# Function for dividing supply between different brands
def merge_supply(brand_number, match_brand_product, match_brand_number, match_product_brand, match_number_brand):
    """This function returns a set of brands' numbers that are produced from the same product"""
    brand_name = match_brand_number[brand_number]
    product_name = match_brand_product[brand_name]
    brands_overlap_names = match_product_brand[product_name]
    brands_overlap_numbers = set(match_number_brand.get(item) for item in brands_overlap_names)

    return brands_overlap_numbers

In [85]:
# Get the list of unique points inside each brand
def extract_unique_points_of_brand(brand):
    points = []
    for product_id in brand.get('product_transportations').keys():
        for transportation in brand.get('product_transportations').get(product_id):
            points.append(transportation.get('point'))

    points = list(set(points))
    
    return points

# Get the list of unique origins inside each brand
def extract_unique_origins_of_brand(brand):
    origins = []
    for transportations in brand.get('product_transportations').values():
        for transportation in transportations:
            origins.append(transportation.get('origin'))

    origins = list(set(origins))
    
    return origins

# Extract the transportation of products from map dictionary
def get_brand_transportations(brand):
    brand_transpotrations = []
    for product_id in brand.get('product_transportations').keys():
        for transportation in brand.get('product_transportations').get(product_id):
            brand_transpotrations.append(transportation)
                
    return brand_transpotrations

# Get the list of unique origins (overall)
def extract_unique_origins(brand_map):
    origins = []
    for brand in brand_map.values():
        origins = origins + extract_unique_origins_of_brand(brand)

    return list(set(origins))

# Get the list of unique points (overall)
def extract_unique_points(brand_map):
    points = []
    for brand in brand_map.values():
        points = points + extract_unique_points_of_brand(brand)
        
    return list(set(points))

# Get transportations of concrete brand
def get_transportations(brand_map):
    transportations = []
    
    for brand in brand_map.values():
        transportations = transportations + get_brand_transportations(brand)
        
    return transportations

In [86]:
def define_problem(df):
    x = [pulp.LpVariable(df['var_Name'][i], lowBound = 0, cat='Continuous') for i in np.arange(0, len(df))]
    # Set function's goal
    problem = pulp.LpProblem('0', sense=LpMinimize)
    # Create a function
    problem += pulp.LpAffineExpression([(x[i], df['cost'][i]) for i in np.arange(0, len(df))])
    x_map = {}
    
    for i in x:
        x_map[str(i)] = i
    
    return problem, x, x_map

In [87]:
# Make restriction on the demand (concrete volume of product needed for point)
def make_restriction_demand(record, problem, demand, x, brands):
    n_constraint = 1
    
    for brand_id in brands:
        brand = record.get(brand_id);
        points = extract_unique_points_of_brand(brand)
    
        for p in points:
            if p in list(real_subset.point.unique()):
                transportations = get_brand_transportations(brand)
                brand_point_transpotrations = [transportation for transportation in transportations if transportation.get('point') == p]
                const = float(demand[(demand['point'] == p) & (demand['brand_number'] == brand_id)]['demand'])
                # Add a restriction on demand == const
                problem+= pulp.LpAffineExpression([(x.get(transportation.get('var_name')), 1) for transportation in brand_point_transpotrations ]) == const, str(n_constraint)
                n_constraint +=1
        
    return problem, n_constraint

In [88]:
# Make restriction on the supply (concrete volume of product available on origin)
# Limitations by month, and by group S, D - upper bound
def make_restriction_supply_by_group_upper(problem, record, bandwidth, supply, x, match_brand_product, match_brand_number, match_product_brand, match_number_brand, brands, match_product_group, match_group_product, match_group_brand, match_brand_group, n_constraint):
    # Set for storing already processed brands 
    brands_accounted_for = set()
    
    # Store origin with infinite supply in a separate list
    #origins_with_inf_supply = list(bandwidth[bandwidth['volume'] == np.inf]['origin'].unique())
    
    for b in brands:
        if b in brands_accounted_for:
            continue
        
        brands_overlap_numbers = merge_supply_by_group(b, match_brand_number, match_number_brand, match_brand_group, match_group_brand)
        brands_accounted_for.update(brands_overlap_numbers)

        brands_list = [brand for brand in record.values() if brand.get('id') in brands_overlap_numbers]

        origins = []
        for brand in brands_list:
            origins += extract_unique_origins_of_brand(brand)
            
        origins = list(set(origins))
        
        transportations_list = []
        for brand in brands_list:
            transportations_list += get_brand_transportations(brand)

        for o in origins:
            
            origin_transportations = [transportation for transportation in transportations_list if transportation.get('origin') == o]
            product_name = match_brand_product[match_brand_number[b]]
            group_name = match_product_group[product_name]
            products_share_group = match_group_product[group_name]
            const = float(supply[(supply['origin'] == o) & (supply['product'].isin(products_share_group))]['supply'].sum())
            
            # Add a restriction on supply <= upper_const
            problem+= pulp.LpAffineExpression([(x.get(i.get('var_name')), 1) for i in origin_transportations ]) <= const, str(n_constraint)
            n_constraint +=1   
                
    return problem, n_constraint

In [89]:
# Make restriction on supply (restriction on lower percentage of loading)
# Limitation by month, group D,and S - only lower bound
def make_restriction_supply_by_group_lower(problem, record, bandwidth, supply, x, match_brand_product, match_brand_number, match_product_brand, match_number_brand, brands, match_product_group, match_group_product, match_group_brand, match_brand_group, n_constraint):
    # Set for storing already processed brands 
    brands_accounted_for = set()
    
    # Store origin with infinite supply in a separate list
    number = 0
    oilbases_dict = dict()

    for i in bandwidth['origin'].unique():
        oilbases_dict[i] = number
        number+=1
        
    origins_with_inf_supply = set(oilbases_dict.values()).difference(constants['ORIGINS_TO_CONTROL'])
    
    for b in brands:
        if b in brands_accounted_for:
            continue
        
        brands_overlap_numbers = merge_supply_by_group(b, match_brand_number, match_number_brand, match_brand_group, match_group_brand)
        brands_accounted_for.update(brands_overlap_numbers)

        brands_list = [brand for brand in record.values() if brand.get('id') in brands_overlap_numbers]

        origins = []
        for brand in brands_list:
            origins += extract_unique_origins_of_brand(brand)
            
        origins = list(set(origins))
        
        transportations_list = []
        for brand in brands_list:
            transportations_list += get_brand_transportations(brand)

        for o in origins:
            
            origin_transportations = [transportation for transportation in transportations_list if transportation.get('origin') == o]

            product_name = match_brand_product[match_brand_number[b]]
            group_name = match_product_group[product_name]
            products_share_group = match_group_product[group_name]
            const = float(supply[(supply['origin'] == o) & (supply['product'].isin(products_share_group))]['supply'].sum())            
            
            if o in origins_with_inf_supply:
                continue       

            else:

                # Add a restriction on supply >= lower_const
                const *= constants['PERCENT_LOADING']
                problem+= pulp.LpAffineExpression([(x.get(i.get('var_name')), 1) for i in origin_transportations ]) >= const, str(n_constraint)
                n_constraint +=1
                    
    return problem, n_constraint

In [90]:
# Limitation by month and product - upper bound
def make_restriction_supply_by_product_upper(problem, record, bandwidth, supply, x, match_brand_product, match_brand_number, match_product_brand, match_number_brand, brands, n_constraint):
    # Set for storing already processed brands 
    brands_accounted_for = set()
    
    # Store origin with infinite supply in a separate list
    #origins_with_inf_supply = list(bandwidth[bandwidth['volume'] == np.inf]['origin'].unique())
    
    for b in brands:
        if b in brands_accounted_for:
            continue
        
        brands_overlap_numbers = merge_supply_by_product(b, match_brand_product, match_brand_number, match_product_brand, match_number_brand)
        brands_accounted_for.update(brands_overlap_numbers)

        brands_list = [brand for brand in record.values() if brand.get('id') in brands_overlap_numbers]

        origins = []
        for brand in brands_list:
            origins += extract_unique_origins_of_brand(brand)
            
        origins = list(set(origins))
        
        transportations_list = []
        for brand in brands_list:
            transportations_list += get_brand_transportations(brand)
            
            
        for o in origins:
            
            origin_transportations = [transportation for transportation in transportations_list if transportation.get('origin') == o]
            product_name = match_brand_product[match_brand_number[b]]
            const = float(supply[(supply['origin'] == o) & (supply['product'] == product_name)]['supply'])
            
            # Add a restriction on supply <= upper_const
            problem+= pulp.LpAffineExpression([(x.get(i.get('var_name')), 1) for i in origin_transportations ]) <= const, str(n_constraint)
            n_constraint +=1

    return problem, n_constraint

In [91]:
# Limitation by month and product - lower bound
def make_restriction_supply_by_product_lower(problem, record, bandwidth, supply, x, match_brand_product, match_brand_number, match_product_brand, match_number_brand, brands, n_constraint):
    # Set for storing already processed brands 
    brands_accounted_for = set()
    
    # Store origin with infinite supply in a separate list
    number = 0
    oilbases_dict = dict()

    for i in bandwidth['origin'].unique():
        oilbases_dict[i] = number
        number+=1
        
    origins_with_inf_supply = set(oilbases_dict.values()).difference(constants['ORIGINS_TO_CONTROL'])
    
    for b in brands:
        if b in brands_accounted_for:
            continue
        
        brands_overlap_numbers = merge_supply_by_product(b, match_brand_product, match_brand_number, match_product_brand, match_number_brand)
        brands_accounted_for.update(brands_overlap_numbers)

        brands_list = [brand for brand in record.values() if brand.get('id') in brands_overlap_numbers]

        origins = []
        for brand in brands_list:
            origins += extract_unique_origins_of_brand(brand)
            
        origins = list(set(origins))
        
        transportations_list = []
        for brand in brands_list:
            transportations_list += get_brand_transportations(brand)

        for o in origins:
            
            origin_transportations = [transportation for transportation in transportations_list if transportation.get('origin') == o]
            product_name = match_brand_product[match_brand_number[b]]
            const = float(supply[(supply['origin'] == o) & (supply['product'] == product_name)]['supply'])
            
            if o in origins_with_inf_supply:
                continue
                
            else:

                # Add a restriction on supply >= lower_const
                const *= constants['PERCENT_LOADING']
                problem+= pulp.LpAffineExpression([(x.get(i.get('var_name')), 1) for i in origin_transportations ]) >= const, str(n_constraint)
                n_constraint +=1 

    return problem, n_constraint

In [92]:
# Make restriction on supply by sum of all products - lower bound
def make_restriction_supply_by_all_products_lower(problem, record, supply, x, brands, n_constraint):
    # Set for storing already processed brands 
    # brands_accounted_for = set()
    
    # Store origin with infinite supply in a separate list
    number = 0
    oilbases_dict = dict()

    for i in bandwidth['origin'].unique():
        oilbases_dict[i] = number
        number+=1
        
    origins_with_inf_supply = set(oilbases_dict.values()).difference(constants['ORIGINS_TO_CONTROL'])
    
    brands_list = [brand for brand in record.values()]
    origins = []
    
    for brand in brands_list:
        origins += extract_unique_origins_of_brand(brand)
            
    origins = list(set(origins))
        
    for o in origins:
        if o in origins_with_inf_supply:
            continue

        else:
            transportations_list = []
            for brand in brands_list:
                transportations_list += get_brand_transportations(brand)            
            
                            
            origin_transportations = [transportation for transportation in transportations_list if transportation.get('origin') == o]

            const = float(supply[(supply['origin'] == o)]['supply'].sum())
            const *= constants['PERCENT_LOADING']
            
            # Add a restriction on supply >= lower_const
            problem+= pulp.LpAffineExpression([(x.get(i.get('var_name')), 1) for i in origin_transportations ]) >= const, str(n_constraint)
            n_constraint +=1
    
    return problem, n_constraint

In [93]:
# Function for creating dummies names
def create_dummy_Name(df):
    dummies = {}
    
    # Create a name variable
    dummies_names = set(df.apply(lambda row: 'y_P' + str(int(row["point"])) + 'O' + str(int(row["origin"])), axis=1))

    # Create dummy variables and store them in a dict
    for name in dummies_names:
        dummies[name] = pulp.LpVariable(name, lowBound=0, upBound = 1, cat = 'Integer')
        
    return dummies

In [94]:
# Create points - origins dictionary
def create_points_dict(df):
    points_dict = {}
    for p in list(df['point'].unique()):
        origins_available = list(df[df['point'] == p]['origin'].unique())
        if points_dict.get(p) == None:
            points_dict[p] = origins_available
    return points_dict

points_dict = create_points_dict(df_reorganized)

In [95]:
# Make restriction on dummy variables (all products are delivered from the same origin to the same point)
def make_restriction_dummy(record, problem, n_constraint, dummies, x, points_dict):
    
    transportations = get_transportations(record)
    
    for p in points_dict.keys():
        
        trans = []
        
        for o in points_dict[p]:
            
            po_transportations = [transportation for transportation in transportations if transportation.get('origin') == o and transportation.get('point') == p]
            
            string = "y_P" + str(p) + "O" + str(o)
            d = dummies.get(string)
            
            # Restriction (...) <= M * Y_PnOm
            problem+= pulp.LpAffineExpression([(x.get(i.get('var_name')), 1) for i in po_transportations ]) <=  constants['M'] * d, str(n_constraint)
            n_constraint += 1
            trans.append(d)
            
        # Restriction (y_PnOm1 + y_PnOm2 + ...) == 1
        problem += pulp.LpAffineExpression([(i, 1) for i in trans]) == 1, str(n_constraint)
        n_constraint += 1  
            
    return problem, n_constraint

### SCIP

In [97]:
scip_start = time.time()

dummies = create_dummy_Name(df_reorganized)

problem_defined, x, x_map = define_problem(df_reorganized)

problem, n_constraint = make_restriction_demand(brand_info_map, problem_defined, demand, x_map, brands)

if constants['UPPER_BOUND'] == 'group':
    problem, n_constraint, = make_restriction_supply_by_group_upper(problem, brand_info_map, bandwidth_subset, supply, x_map, match_brand_product, match_brand_number, match_product_brand, match_number_brand, brands, match_product_group, match_group_product, match_group_brand, match_brand_group, n_constraint)
elif constants['UPPER_BOUND'] == 'product':
    problem, n_constraint = make_restriction_supply_by_product_upper(problem, brand_info_map, bandwidth_subset, supply, x_map, match_brand_product, match_brand_number, match_product_brand, match_number_brand, brands, n_constraint)

if constants['LOWER_BOUND'] == 'group':
    problem, n_constraint = make_restriction_supply_by_group_lower(problem, brand_info_map, bandwidth_subset, supply, x_map, match_brand_product, match_brand_number, match_product_brand, match_number_brand, brands, match_product_group, match_group_product, match_group_brand, match_brand_group, n_constraint)
elif constants['LOWER_BOUND'] == 'product':
    problem, n_constraint = make_restriction_supply_by_product_lower(problem, brand_info_map, bandwidth_subset, supply, x, match_brand_product, match_brand_number, match_product_brand, match_number_brand, brands, n_constraint)
elif constants['LOWER_BOUND'] == 'all products':
    problem, n_constraint = make_restriction_supply_by_all_products_lower(problem, brand_info_map, supply, x_map, brands, n_constraint)
    
problem, n_constraint = make_restriction_dummy(brand_info_map, problem, n_constraint, dummies, x_map, points_dict)

scip_end = time.time()
print('Время работы SCIP: ', scip_end-scip_start)

Время работы SCIP:  11.718956232070923


In [45]:
# def main(problem):
#     start = time.time()
#     status = problem.solve(SCIP_CMD())
    
#     print(status)
#     # Print the results
#     print ("Результат:")
#     for variable in problem.variables():
#         print (variable.name, "=", variable.varValue)
#     print ("Стоимость доставки:")
#     print (abs(problem.objective.value()))
#     stop = time.time()
#     print ("Время :")
#     print(stop - start)

In [46]:
# # This is SCIP_CMD
# if __name__ == '__main__':
#     main(problem)
#     scip_end = time.time()
#     print('Время работы SCIP: ', scip_end)

### CBC

In [98]:
cbc_start = time.time()

dummies = create_dummy_Name(df_reorganized)

problem_defined, x, x_map = define_problem(df_reorganized)

problem, n_constraint = make_restriction_demand(brand_info_map, problem_defined, demand, x_map, brands)

if constants['UPPER_BOUND'] == 'group':
    problem, n_constraint, = make_restriction_supply_by_group_upper(problem, brand_info_map, bandwidth_subset, supply, x_map, match_brand_product, match_brand_number, match_product_brand, match_number_brand, brands, match_product_group, match_group_product, match_group_brand, match_brand_group, n_constraint)
elif constants['UPPER_BOUND'] == 'product':
    problem, n_constraint = make_restriction_supply_by_product_upper(problem, brand_info_map, bandwidth_subset, supply, x_map, match_brand_product, match_brand_number, match_product_brand, match_number_brand, brands, n_constraint)

if constants['LOWER_BOUND'] == 'group':
    problem, n_constraint = make_restriction_supply_by_group_lower(problem, brand_info_map, bandwidth_subset, supply, x_map, match_brand_product, match_brand_number, match_product_brand, match_number_brand, brands, match_product_group, match_group_product, match_group_brand, match_brand_group, n_constraint)
elif constants['LOWER_BOUND'] == 'product':
    problem, n_constraint = make_restriction_supply_by_product_lower(problem, brand_info_map, bandwidth_subset, supply, x, match_brand_product, match_brand_number, match_product_brand, match_number_brand, brands, n_constraint)
elif constants['LOWER_BOUND'] == 'all products':
    problem, n_constraint = make_restriction_supply_by_all_products_lower(problem, brand_info_map, supply, x_map, brands, n_constraint)
    
problem, n_constraint = make_restriction_dummy(brand_info_map, problem, n_constraint, dummies, x_map, points_dict)

cbc_end = time.time()
print('Время введения ограничений: ', cbc_end-cbc_start)

Время работы CBC:  10.773516178131104


In [51]:
def main(problem):
    start = time.time()
    status = problem.solve(PULP_CBC_CMD(timeLimit=60*20, warmStart=True))
    
    print(status)
    # Print the results
    print ("Результат:")
    for variable in problem.variables():
        print (variable.name, "=", variable.varValue)
    print ("Стоимость доставки:")
    print (abs(problem.objective.value()))
    stop = time.time()
    print ("Время :")
    print(stop - start)

In [52]:
# # This is PULP_CBC_CMD
# if __name__ == '__main__':
#     main(problem)
#     cbc_end = time.time()
#     print('Время работы солвера CBC: ', cbc_end-cbc_start)

1
Результат:
P100O10_brand_1 = 186.7181
P100O10_brand_2 = 330.5634
P100O10_brand_3 = 93.9749
P100O10_brand_5 = 158.9166
P101O10_brand_1 = 143.3758
P101O10_brand_2 = 149.009
P101O10_brand_3 = 45.6636
P101O10_brand_7 = 87.9395
P102O10_brand_1 = 0.0
P102O10_brand_2 = 0.0
P102O10_brand_3 = 0.0
P102O10_brand_5 = 0.0
P102O13_brand_1 = 0.0
P102O13_brand_2 = 0.0
P102O13_brand_3 = 0.0
P102O13_brand_5 = 0.0
P102O19_brand_1 = 152.4403
P102O19_brand_2 = 111.9759
P102O19_brand_3 = 30.3199
P102O19_brand_5 = 137.7352
P103O10_brand_1 = 60.9802
P103O10_brand_2 = 71.7035
P103O10_brand_3 = 19.0324
P103O10_brand_5 = 274.0885
P104O10_brand_1 = 93.9808
P104O10_brand_2 = 77.6811
P104O10_brand_3 = 15.7107
P104O10_brand_5 = 140.1428
P105O10_brand_1 = 78.1203
P105O10_brand_2 = 102.6877
P105O10_brand_3 = 24.4948
P105O10_brand_5 = 122.7953
P106O10_brand_1 = 236.547
P106O10_brand_3 = 261.7828
P106O10_brand_5 = 152.98
P106O10_brand_6 = 8.7218
P106O10_brand_7 = 107.1475
P107O10_brand_1 = 137.1438
P107O10_brand_2 = 1

P132O13_brand_5 = 0.0
P132O13_brand_6 = 0.0
P132O19_brand_1 = 234.6897
P132O19_brand_2 = 406.4215
P132O19_brand_3 = 142.0962
P132O19_brand_5 = 212.3869
P132O19_brand_6 = 57.4376
P133O13_brand_1 = 0.0
P133O13_brand_2 = 0.0
P133O13_brand_3 = 0.0
P133O13_brand_5 = 0.0
P133O13_brand_6 = 0.0
P133O19_brand_1 = 143.8142
P133O19_brand_2 = 205.4954
P133O19_brand_3 = 83.3423
P133O19_brand_5 = 154.5429
P133O19_brand_6 = 24.6367
P134O13_brand_1 = 0.0
P134O13_brand_2 = 0.0
P134O13_brand_3 = 0.0
P134O13_brand_5 = 0.0
P134O13_brand_6 = 0.0
P134O19_brand_1 = 281.6913
P134O19_brand_2 = 246.3147
P134O19_brand_3 = 67.8137
P134O19_brand_5 = 272.6964
P134O19_brand_6 = 20.5515
P135O13_brand_1 = 0.0
P135O13_brand_2 = 0.0
P135O13_brand_3 = 0.0
P135O13_brand_5 = 0.0
P135O19_brand_1 = 189.8187
P135O19_brand_2 = 212.609
P135O19_brand_3 = 51.1988
P135O19_brand_5 = 124.4193
P136O13_brand_1 = 0.0
P136O13_brand_3 = 0.0
P136O13_brand_5 = 0.0
P136O13_brand_6 = 0.0
P136O19_brand_1 = 361.5905
P136O19_brand_3 = 419.5659


P185O19_brand_6 = 15.8948
P186O13_brand_1 = 0.0
P186O13_brand_2 = 0.0
P186O13_brand_3 = 0.0
P186O13_brand_5 = 0.0
P186O13_brand_6 = 0.0
P186O19_brand_1 = 88.6486
P186O19_brand_2 = 119.3214
P186O19_brand_3 = 46.8803
P186O19_brand_5 = 117.9561
P186O19_brand_6 = 21.9659
P187O13_brand_1 = 0.0
P187O13_brand_2 = 0.0
P187O13_brand_3 = 0.0
P187O13_brand_5 = 0.0
P187O19_brand_1 = 108.9736
P187O19_brand_2 = 150.3787
P187O19_brand_3 = 34.95
P187O19_brand_5 = 121.6729
P188O13_brand_1 = 0.0
P188O13_brand_2 = 0.0
P188O13_brand_3 = 0.0
P188O13_brand_5 = 0.0
P188O13_brand_6 = 0.0
P188O19_brand_1 = 172.904
P188O19_brand_2 = 277.2507
P188O19_brand_3 = 110.1205
P188O19_brand_5 = 261.7518
P188O19_brand_6 = 42.5015
P189O13_brand_1 = 245.5835
P189O13_brand_2 = 272.6391
P189O13_brand_3 = 84.3228
P189O13_brand_5 = 423.2214
P189O13_brand_6 = 27.1317
P189O19_brand_1 = 0.0
P189O19_brand_2 = 0.0
P189O19_brand_3 = 0.0
P189O19_brand_5 = 0.0
P189O19_brand_6 = 0.0
P18O10_brand_1 = 115.0513
P18O10_brand_2 = 132.7518
P

P244O19_brand_1 = 201.1094
P244O19_brand_2 = 177.2261
P244O19_brand_3 = 39.4895
P244O19_brand_5 = 459.678
P244O19_brand_6 = 8.0899
P245O13_brand_1 = 237.6364
P245O13_brand_2 = 282.3845
P245O13_brand_3 = 60.8602
P245O13_brand_5 = 338.3817
P245O13_brand_6 = 13.0499
P245O19_brand_1 = 0.0
P245O19_brand_2 = 0.0
P245O19_brand_3 = 0.0
P245O19_brand_5 = 0.0
P245O19_brand_6 = 0.0
P246O13_brand_1 = 0.0
P246O13_brand_2 = 0.0
P246O13_brand_3 = 0.0
P246O13_brand_5 = 0.0
P246O13_brand_6 = 0.0
P246O19_brand_1 = 169.0322
P246O19_brand_2 = 190.1038
P246O19_brand_3 = 37.0359
P246O19_brand_5 = 271.3377
P246O19_brand_6 = 14.0098
P247O13_brand_1 = 0.0
P247O13_brand_2 = 0.0
P247O13_brand_3 = 0.0
P247O13_brand_5 = 0.0
P247O13_brand_6 = 0.0
P247O19_brand_1 = 151.4889
P247O19_brand_2 = 99.8805
P247O19_brand_3 = 17.7533
P247O19_brand_5 = 81.4641
P247O19_brand_6 = 2.217
P248O13_brand_0 = 0.0
P248O13_brand_1 = 0.0
P248O13_brand_2 = 0.0
P248O13_brand_3 = 0.0
P248O13_brand_5 = 0.0
P248O19_brand_0 = 19.2796
P248O19_

P292O19_brand_2 = 169.3022
P292O19_brand_3 = 55.8465
P292O19_brand_5 = 850.7658
P292O19_brand_6 = 16.9269
P292O9_brand_1 = 0.0
P292O9_brand_2 = 0.0
P292O9_brand_3 = 0.0
P292O9_brand_5 = 0.0
P292O9_brand_6 = 0.0
P293O12_brand_1 = 0.0
P293O12_brand_2 = 0.0
P293O12_brand_3 = 0.0
P293O12_brand_5 = 0.0
P293O13_brand_1 = 0.0
P293O13_brand_2 = 0.0
P293O13_brand_3 = 0.0
P293O13_brand_5 = 0.0
P293O13_brand_6 = 0.0
P293O19_brand_1 = 89.0259
P293O19_brand_2 = 105.4887
P293O19_brand_3 = 37.8224
P293O19_brand_5 = 835.6082
P293O19_brand_6 = 10.763
P293O9_brand_1 = 0.0
P293O9_brand_2 = 0.0
P293O9_brand_3 = 0.0
P293O9_brand_5 = 0.0
P293O9_brand_6 = 0.0
P294O13_brand_1 = 0.0
P294O13_brand_3 = 0.0
P294O13_brand_5 = 0.0
P294O13_brand_6 = 0.0
P294O19_brand_1 = 178.3421
P294O19_brand_3 = 273.5011
P294O19_brand_5 = 1058.8739
P294O19_brand_6 = 18.7603
P295O13_brand_1 = 393.8594
P295O13_brand_2 = 174.5762
P295O13_brand_3 = 187.4139
P295O13_brand_5 = 358.7859
P295O13_brand_6 = 11.3467
P295O19_brand_1 = 0.0
P29

...

P429O13_brand_2 = 0.0
P429O13_brand_3 = 0.0
P429O13_brand_5 = 0.0
P429O19_brand_1 = 0.0
P429O19_brand_2 = 0.0
P429O19_brand_3 = 0.0
P429O19_brand_5 = 0.0
P429O9_brand_1 = 0.0
P429O9_brand_2 = 0.0
P429O9_brand_3 = 0.0
P429O9_brand_5 = 0.0
P42O10_brand_1 = 93.3504
P42O10_brand_2 = 63.6923
P42O10_brand_3 = 67.7672
P42O10_brand_5 = 99.2107
P42O10_brand_6 = 4.9349
P430O12_brand_1 = 0.0
P430O12_brand_2 = 0.0
P430O12_brand_5 = 0.0
P430O13_brand_1 = 0.0
P430O13_brand_2 = 0.0
P430O13_brand_5 = 0.0
P430O19_brand_1 = 102.3243
P430O19_brand_2 = 94.3944
P430O19_brand_5 = 249.7712
P430O9_brand_1 = 0.0
P430O9_brand_2 = 0.0
P430O9_brand_5 = 0.0
P431O12_brand_1 = 0.0
P431O12_brand_2 = 0.0
P431O12_brand_3 = 0.0
P431O12_brand_5 = 0.0
P431O13_brand_1 = 0.0
P431O13_brand_2 = 0.0
P431O13_brand_3 = 0.0
P431O13_brand_5 = 0.0
P431O13_brand_6 = 0.0
P431O19_brand_1 = 87.9614
P431O19_brand_2 = 109.8248
P431O19_brand_3 = 19.0032
P431O19_brand_5 = 894.9694
P431O19_brand_6 = 4.1387
P431O9_brand_1 = 0.0
P431O9_brand_

P466O12_brand_5 = 0.0
P466O13_brand_1 = 0.0
P466O13_brand_2 = 0.0
P466O13_brand_5 = 0.0
P466O13_brand_6 = 0.0
P466O19_brand_1 = 0.0
P466O19_brand_2 = 0.0
P466O19_brand_5 = 0.0
P466O19_brand_6 = 0.0
P466O9_brand_1 = 125.0572
P466O9_brand_2 = 141.3722
P466O9_brand_5 = 71.1094
P466O9_brand_6 = 7.1949
P467O12_brand_1 = 139.5588
P467O12_brand_2 = 97.4164
P467O12_brand_3 = 13.524
P467O12_brand_5 = 247.9027
P467O13_brand_1 = 0.0
P467O13_brand_2 = 0.0
P467O13_brand_3 = 0.0
P467O13_brand_5 = 0.0
P467O19_brand_1 = 0.0
P467O19_brand_2 = 0.0
P467O19_brand_3 = 0.0
P467O19_brand_5 = 0.0
P467O9_brand_1 = 0.0
P467O9_brand_2 = 0.0
P467O9_brand_3 = 0.0
P467O9_brand_5 = 0.0
P468O12_brand_1 = 0.0
P468O12_brand_2 = 0.0
P468O12_brand_3 = 0.0
P468O12_brand_5 = 0.0
P468O13_brand_1 = 0.0
P468O13_brand_2 = 0.0
P468O13_brand_3 = 0.0
P468O13_brand_5 = 0.0
P468O13_brand_6 = 0.0
P468O19_brand_1 = 0.0
P468O19_brand_2 = 0.0
P468O19_brand_3 = 0.0
P468O19_brand_5 = 0.0
P468O19_brand_6 = 0.0
P468O9_brand_1 = 47.9544
P46

P519O13_brand_1 = 0.0
P519O13_brand_2 = 0.0
P519O13_brand_3 = 0.0
P519O13_brand_9 = 0.0
P519O19_brand_0 = 16.4925
P519O19_brand_1 = 245.8397
P519O19_brand_2 = 229.6125
P519O19_brand_3 = 60.0377
P519O19_brand_9 = 367.8248
P519O26_brand_1 = 0.0
P519O26_brand_2 = 0.0
P519O26_brand_3 = 0.0
P519O26_brand_9 = 0.0
P519O28_brand_0 = 0.0
P519O28_brand_1 = 0.0
P519O28_brand_2 = 0.0
P519O28_brand_3 = 0.0
P519O28_brand_9 = 0.0
P519O29_brand_0 = 0.0
P519O29_brand_1 = 0.0
P519O29_brand_2 = 0.0
P519O29_brand_3 = 0.0
P519O29_brand_9 = 0.0
P51O10_brand_1 = 110.8947
P51O10_brand_2 = 105.7159
P51O10_brand_3 = 24.0796
P51O10_brand_5 = 198.4539
P520O13_brand_1 = 0.0
P520O13_brand_2 = 0.0
P520O13_brand_3 = 0.0
P520O19_brand_1 = 209.7031
P520O19_brand_2 = 228.195
P520O19_brand_3 = 69.2304
P520O26_brand_1 = 0.0
P520O26_brand_2 = 0.0
P520O26_brand_3 = 0.0
P520O28_brand_1 = 0.0
P520O28_brand_2 = 0.0
P520O28_brand_3 = 0.0
P520O29_brand_1 = 0.0
P520O29_brand_2 = 0.0
P520O29_brand_3 = 0.0
P521O13_brand_0 = 0.0
P52

P90O10_brand_6 = 11.4915
P91O10_brand_1 = 66.1624
P91O10_brand_2 = 77.503
P91O10_brand_3 = 23.3889
P91O10_brand_5 = 147.9714
P92O10_brand_1 = 268.0289
P92O10_brand_3 = 318.7677
P92O10_brand_6 = 19.0319
P92O10_brand_7 = 130.3726
P93O10_brand_1 = 247.8817
P93O10_brand_2 = 223.8465
P93O10_brand_3 = 56.0468
P93O10_brand_7 = 101.834
P94O10_brand_1 = 186.8134
P94O10_brand_2 = 230.6322
P94O10_brand_3 = 54.0577
P94O10_brand_5 = 312.1056
P95O10_brand_1 = 129.7241
P95O10_brand_2 = 135.127
P95O10_brand_3 = 30.8544
P95O10_brand_5 = 257.4422
P96O10_brand_1 = 161.9148
P96O10_brand_2 = 174.0933
P96O10_brand_3 = 43.9527
P96O10_brand_5 = 156.4785
P97O10_brand_1 = 140.507
P97O10_brand_2 = 211.4792
P97O10_brand_3 = 78.5684
P97O10_brand_7 = 94.9734
P98O10_brand_1 = 128.5366
P98O10_brand_2 = 196.0517
P98O10_brand_3 = 46.1371
P98O10_brand_5 = 99.4259
P99O10_brand_1 = 70.3168
P99O10_brand_2 = 127.8197
P99O10_brand_3 = 29.2319
P99O10_brand_5 = 60.5798
P9O10_brand_1 = 227.4278
P9O10_brand_2 = 296.1759
P9O10_br

y_P304O19 = 0.0
y_P305O13 = 1.0
y_P305O19 = 0.0
y_P306O13 = 1.0
y_P306O19 = 0.0
y_P307O13 = 0.0
y_P307O19 = 1.0
y_P308O13 = 0.0
y_P308O19 = 1.0
y_P309O13 = 0.0
y_P309O19 = 1.0
y_P30O10 = 1.0
y_P310O13 = 0.0
y_P310O19 = 1.0
y_P311O13 = 0.0
y_P311O19 = 1.0
y_P312O13 = 0.0
y_P312O19 = 1.0
y_P313O13 = 1.0
y_P313O19 = 0.0
y_P314O13 = 1.0
y_P314O19 = 0.0
y_P315O13 = 0.0
y_P315O19 = 1.0
y_P316O13 = 1.0
y_P316O19 = 0.0
y_P317O13 = 1.0
y_P317O19 = 0.0
y_P318O13 = 1.0
y_P318O19 = 0.0
y_P319O13 = 0.0
y_P319O19 = 1.0
y_P31O10 = 1.0
y_P320O13 = 1.0
y_P320O19 = 0.0
y_P321O13 = 1.0
y_P321O19 = 0.0
y_P322O13 = 1.0
y_P322O19 = 0.0
y_P323O13 = 0.0
y_P323O19 = 1.0
y_P324O13 = 1.0
y_P324O19 = 0.0
y_P325O13 = 1.0
y_P325O19 = 0.0
y_P326O13 = 0.0
y_P326O19 = 1.0
y_P327O13 = 0.0
y_P327O19 = 1.0
y_P328O13 = 0.0
y_P328O19 = 1.0
y_P329O13 = 0.0
y_P329O19 = 1.0
y_P32O10 = 1.0
y_P330O13 = 0.0
y_P330O19 = 1.0
y_P331O13 = 1.0
y_P331O19 = 0.0
y_P332O13 = 1.0
y_P332O19 = 0.0
y_P333O13 = 0.0
y_P333O19 = 1.0
y_P334O13 =